# Azure OpenAI Responses API Sample

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables


file_path = os.environ.get("TARGET_FILE_PATH")
model_name = os.environ.get("MODEL_NAME", "o3-pro")

system_prompt ='''
あなたは優秀な医者です。
'''

user_content = '''
与えられた心電図（ECG）データを分析し、以下の情報を提供してください：
1. 心拍数（bpm）    
2. 心拍のリズム（正常、頻脈、徐脈など）
3. 異常な波形の有無（例：心房細動、心室性期外収縮など）
4. その他の重要な所見
'''

api_version = "2025-04-01-preview"
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")


In [ ]:
from IPython.display import Image, display

import os

import base64
f = open(file_path, 'rb')
data = f.read()
image_data = base64.b64encode(data).decode("utf-8")
image_format = os.environ.get("FILE_SUFFIX", "jpg")
data_url = f"data:image/{image_format};base64,{image_data}"
f.close()
display(Image(data))


In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import httpx

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    timeout=httpx.Timeout(3600.0, read=3600.0, write=3600.0, connect=3600.0)
)

In [ ]:
response = client.responses.create(
    model= model_name,
    instructions="回答は日本語ですること",
    input=[
        {
            "role": "system",
            "content":  {
                "type": "text",
                "text": system_prompt
            },
            "role": "user",
            "content": [ 
                {"type": "input_text", "text": user_content},
                {"type": "input_image", "image_url": data_url}
                ]
            }
    ]
)


In [ ]:
print(f"model:{response.model}")
print(f"output:\n{response.output[1].content[0].text}")
print(f"input_tokens:{response.usage.input_tokens}\noutput_tokens:{response.usage.output_tokens}\ntotal_tokens:{response.usage.total_tokens}")